## This notebook shows the way we got the figures in our article.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from pt.server import Server
import time
from tools.utils import *
import shutil
import os
import glob
from tqdm.notebook import tqdm

degrees = [90, 180]
masks = [10] + list(range(50, 641, 50)) + [640] 
mkdirs('pt/log/plot')

In [ ]:
'''
Prepare configs
'''
mkdirs('configs/plot')

yaml_write(yaml_read('configs/cfg.yml'), 'configs/plot/yolo_v5.yml')

def new_cfg_ro(list_degree, list_mask):
    for degree in list_degree:
        temp = yaml_read('configs/cfg_ro.yml')
        temp['n_degree'] = degree
        for mask in list_mask:
            temp['nms']['mask_h'] = mask
            temp['nms']['mask_w'] = mask
            yaml_write(temp, f'configs/plot/ro_yolo_{degree}_{mask}.yml')
new_cfg_ro(degrees, masks)


In [ ]:
'''
Prepare FPS|__mask_size
'''
res = {}
for degree in degrees:
    for mask in masks:
        record = Record()
        server = Server(cfg_path=f'configs/plot/ro_yolo_{degree}_{mask}.yml',
                        infer_only=True,
                        infer_model=f'pt/log/robndbox/degree_{degree}/best.pt')
        # You need to specify path to your dataset
        f_list = glob.glob('../drug_det_data_gen/data/dataset_eval_latex/yolo/ro_images/*')
        for f_img in tqdm(f_list,
                         desc=f'Calc Frames Per Second, degree={degree}, mask_size={mask}'):
            s_time = time.time()
            pred = server.single_infer(f_img)
            record.update(time = time.time() - s_time)
        res[f'{degree}_{mask}'] = 1/(record.get_record()['time'])
json_write(res, 'pt/log/plot/fps_masks.json')

In [ ]:
'''
Prepare mAP|__mask_size
'''
res = {}
for degree in degrees:
    for mask in masks:
        print(f'Calc mAP, degree={degree}, mask_size={mask}')
        server = Server(cfg_path=f'configs/plot/ro_yolo_{degree}_{mask}.yml',
                        infer_only=True,
                        infer_model=f'pt/log/robndbox/degree_{degree}/best.pt')
        # You need to specify path to your dataset
        metric = server.evaluate('../drug_det_data_gen/data/dataset_eval_latex/yolo/ro_images/', 
                                 '../drug_det_data_gen/data/dataset_eval_latex/yolo/ro_labels/', 
                                 'data/dataset_eval_latex_robndbox/', 
                                 )
        res[f'{degree}_{mask}'] = metric.mAP
json_write(res, 'pt/log/plot/map_masks.json')

In [ ]:
'''
Plot FPS|__mask_size, mAP|__mask_size
'''
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(8,4), sharex=True)
fig.subplots_adjust(left=0.04, right=0.96, bottom=0.06, top=0.94,
                        hspace=0.08, wspace=0.2)
ax = axs[0]
fps = json_read('pt/log/plot/fps_masks.json')
ax.plot(masks, [fps[f'90_{i}'] for i in masks], marker='s', markersize=10, label='Ro_Yolo_90',linewidth=3)
ax.plot(masks, [fps[f'180_{i}'] for i in masks], marker='^', markersize=10, label='Ro_Yolo_180',linewidth=3)
plt.xlabel('Mask size', size=18,labelpad=0)
ax.set_ylabel('FPS', size=18, labelpad=0)
ax.tick_params(labelsize=14,direction='in',pad=2.5)
plt.xlim(0,650)
ax.set_ylim(0.5,4.5)
ax.legend(fontsize=14, framealpha=1)
ax.grid(alpha=0.5)
ax.vlines(550,0,6,colors='r',linestyles='--',linewidth=1.2)

ax = axs[1]
mAP = json_read('pt/log/plot/map_masks.json')
ax.plot(masks, [mAP[f'90_{i}'] for i in masks], marker='s', markersize=10, label='Ro_Yolo_90',linewidth=3)
ax.plot(masks, [mAP[f'180_{i}'] for i in masks], marker='^', markersize=10, label='Ro_Yolo_180',linewidth=3)
ax.set_ylabel('mAP', size=18, labelpad=0)
ax.tick_params(labelsize=14,direction='in',pad=2.5)
ax.set_ylim(0.35,0.9)
ax.grid(alpha=0.5)
ax.vlines(550,0,1,colors='r',linestyles='--',linewidth=1.2)

plt.savefig('pt/log/plot/fps_map_masks.png', dpi=200, bbox_inches = 'tight')
shutil.copy('pt/log/plot/fps_map_masks.png', '../../drug_det_trip/paper_qims_0/images/')

In [ ]:
'''
Prepare figure mAP|__num of parameters
'''
res = {}

server = Server(cfg_path='configs/cfg.yml',
                infer_only=True,
                infer_model='pt/log/bndbox/best.pt')
res['num_param_yolo_v5'] = float(calc_param_size(server.model.parameters()).split(': ')[-1].split(' M')[0])
# You need to specify path to your dataset
metric = server.evaluate('../drug_det_data_gen/data/dataset_eval_latex/yolo/images/', 
                         '../drug_det_data_gen/data/dataset_eval_latex/yolo/labels/', 
                         'data/dataset_eval_bndbox/', 
                         )
res['mAP_yolo_v5'] = metric.mAP

record = Record()
# You need to specify path to your dataset
f_list = glob.glob('../drug_det_data_gen/data/dataset_eval_latex/yolo/ro_images/*')
for f_img in tqdm(f_list,
                 desc=f'Calc Frames Per Second, yolo_v5'):
    s_time = time.time()
    pred = server.single_infer(f_img)
    record.update(time = time.time() - s_time)
res['fps_yolo_v5'] = 1/(record.get_record()['time'])

masks = [500, 550, 600, 640]
for degree in degrees:
    for mask in masks:
        server = Server(cfg_path=f'configs/plot/ro_yolo_{degree}_{mask}.yml',
                        infer_only=True,
                        infer_model=f'pt/log/robndbox/degree_{degree}/best.pt')
        res[f'num_param_ro_yolo_{degree}_{mask}'] = float(calc_param_size(server.model.parameters()).split(': ')[-1].split(' M')[0])
        record = Record()
        # You need to specify path to your dataset
        f_list = glob.glob('../drug_det_data_gen/data/dataset_eval_latex/yolo/ro_images/*')
        for f_img in tqdm(f_list,
                         desc=f'Calc Frames Per Second, degree={degree}, mask_size={mask}'):
            s_time = time.time()
            pred = server.single_infer(f_img)
            record.update(time = time.time() - s_time)
        res[f'fps_ro_yolo_{degree}_{mask}'] = 1/(record.get_record()['time'])

        print(f'Calc mAP, degree={degree}, mask_size={mask}')
        # You need to specify path to your dataset
        metric = server.evaluate('../drug_det_data_gen/data/dataset_eval_latex/yolo/ro_images/', 
                                 '../drug_det_data_gen/data/dataset_eval_latex/yolo/ro_labels/', 
                                 'data/dataset_eval_latex_robndbox/', 
                                 )
        res[f'mAP_ro_yolo_{degree}_{mask}'] = metric.mAP
    
json_write(res, 'pt/log/plot/map_nparam_fps.json')

In [ ]:
'''
Plot mAP|__Num of Param, mAP|__FPS
'''

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(8,3), sharey=True)
fig.subplots_adjust(left=0.04, right=0.96, bottom=0.06, top=0.94,
                        hspace=0.08, wspace=0.05)
ms = 80

ms_plot = 8
masks = [500, 550, 640]
lw = 2

ax = axs[0]
res = json_read('pt/log/plot/map_nparam_fps.json')
ax.scatter(res['num_param_yolo_v5'], res['mAP_yolo_v5'], marker='.', s=ms*4, label='Yolo-v5', c='g')
ax.plot([res[f'num_param_ro_yolo_90_{mask}'] for mask in masks], 
        [res[f'mAP_ro_yolo_90_{mask}'] for mask in masks] , 
        marker='s', markersize=ms_plot, label='Ro_Yolo_90', linewidth=lw)
ax.plot([res[f'num_param_ro_yolo_180_{mask}'] for mask in masks], 
        [res[f'mAP_ro_yolo_180_{mask}'] for mask in masks] , 
           marker='^', markersize=ms_plot, label='Ro_Yolo_180', linewidth=lw)

ax.set_xlabel('Number of parameters (M)', size=15,labelpad=0)
ax.set_ylabel('mAP', size=15, labelpad=0)
ax.tick_params(labelsize=14,direction='in',pad=2.5)
# ax.set_xlim(7,7.75)
# ax.set_ylim(0.735,0.825)
ax.legend(fontsize=12, framealpha=1)
ax.grid(alpha=0.5)


offset_x = 0.02
offset_y = -0.001
for mask in masks:
    x = res[f'num_param_ro_yolo_90_{mask}'] 
    y = res[f'mAP_ro_yolo_90_{mask}']
    ax.text(x+offset_x, y+offset_y, r'$h_{\rm{m}}=w_{\rm{m}}=$'+str(mask)+ (r'$\spadesuit$' if mask == 550 else ''),
             color='black', weight='roman',
             size=11,)

    
offset_x = -0.17
offset_y = -0.002
for mask in masks:
    x = res[f'num_param_ro_yolo_180_{mask}'] 
    y = res[f'mAP_ro_yolo_180_{mask}']
    ax.text(x+offset_x, y+offset_y, r'$h_{\rm{m}}=w_{\rm{m}}=$'+str(mask),
             color='black', weight='roman',
             size=11,)
    if mask == 550:
        ax.text(x+offset_x-0.02, y+offset_y, r'$\spadesuit$',
                 color='black', weight='roman',
                 size=11,)


ax = axs[1]
ax.scatter(res['fps_yolo_v5'], res['mAP_yolo_v5'], marker='.', s=ms*4, label='yolo_v5', c='g')
ax.plot([res[f'fps_ro_yolo_90_{mask}'] for mask in masks], 
        [res[f'mAP_ro_yolo_90_{mask}'] for mask in masks] , 
        marker='s', markersize=ms_plot, label='ro_yolo_90', linewidth=lw)
ax.plot([res[f'fps_ro_yolo_180_{mask}'] for mask in masks], 
        [res[f'mAP_ro_yolo_180_{mask}'] for mask in masks] , 
           marker='^', markersize=ms_plot, label='ro_yolo_180', linewidth=lw)
ax.tick_params(labelsize=14,direction='in',pad=2.5)
ax.grid(alpha=0.5)
# ax.set_xlim(2.5,9)
ax.set_xlabel('FPS', size=15,labelpad=0)

offset_x = 0.2
offset_y = -0.001
for mask in masks:
    x = res[f'fps_ro_yolo_90_{mask}'] 
    y = res[f'mAP_ro_yolo_90_{mask}']
    if mask == 640:
        x += 0.15
    ax.text(x+offset_x, y+offset_y, r'$h_{\rm{m}}=w_{\rm{m}}=$'+str(mask) + (r'$\spadesuit$' if mask == 550 else ''),
             color='black', weight='roman',
             size=11,)
    
offset_x = 0.2
offset_y = -0.0015
for mask in masks:
    x = res[f'fps_ro_yolo_180_{mask}'] 
    y = res[f'mAP_ro_yolo_180_{mask}']
    if mask == 640:
        x += 0.02
    ax.text(x+offset_x, y+offset_y, r'$h_{\rm{m}}=w_{\rm{m}}=$'+str(mask) + (r'$\spadesuit$' if mask == 550 else ''),
             color='black', weight='roman',
             size=11,)

plt.savefig('pt/log/plot/map_nparam_fps.png',dpi=200, bbox_inches = 'tight')
shutil.copy('pt/log/plot/map_nparam_fps.png', '../../drug_det_trip/paper_qims_0/images/')

In [ ]:
'''
Prepare for table
'''

server = Server(cfg_path=f'configs/plot/ro_yolo_180_550.yml',
                infer_only=True,
                infer_model=f'pt/log/robndbox/degree_180/best.pt')
# You need to specify path to your dataset
metric_180 = server.evaluate('../drug_det_data_gen/data/dataset_eval_latex/yolo/ro_images/', 
                         '../drug_det_data_gen/data/dataset_eval_latex/yolo/ro_labels/', 
                         'data/dataset_eval_latex_robndbox/'
                         )

server = Server(cfg_path=f'configs/plot/ro_yolo_90_550.yml',
                infer_only=True,
                infer_model=f'pt/log/robndbox/degree_90/best.pt')
# You need to specify path to your dataset
metric_90 = server.evaluate('../drug_det_data_gen/data/dataset_eval_latex/yolo/ro_images/', 
                         '../drug_det_data_gen/data/dataset_eval_latex/yolo/ro_labels/', 
                         'data/dataset_eval_latex_robndbox/'
                         )

server = Server(cfg_path=f'configs/plot/yolo_v5.yml',
                infer_only=True,
                infer_model=f'pt/log/bndbox/best.pt')
# You need to specify path to your dataset
metric_v5 = server.evaluate('../drug_det_data_gen/data/dataset_eval_latex/yolo/images/train2017/', 
                         '../drug_det_data_gen/data/dataset_eval_latex/yolo/labels/train2017/', 
                         'data/dataset_eval_latex_bndbox/'
                         )


In [ ]:
'''
Show table
'''
names = ['metric_v5', 'metric_90', 'metric_180']
show = lambda x: round(x, 3)
for i, metric in enumerate([metric_v5, metric_90, metric_180]):
    print(f'{names[i]}, mAP = {show(metric.mAP)}; AP_50 = {show(metric.mAPs[0.5])};' + 
          f' AP_75 = {show(metric.mAPs[0.75])}; AP_95 = {show(metric.mAPs[0.95])};' +
          f' Precision = {show(np.mean(metric.precision[-1]))}; Recall = {show(np.mean(metric.recall[-1]))};' +
          f' F1 = {show(np.mean(metric.f1[-1]))}.')